# Structured Q&A

Source code: https://github.com/mozilla-ai/structured-qa

Docs: https://mozilla-ai.github.io/structured-qa

## GPU Check

First, you'll need to enable GPUs for the notebook:

- Navigate to `Edit`→`Notebook Settings`
- Select T4 GPU from the Hardware Accelerator section
- Click `Save` and accept.

Next, we'll confirm that we can connect to the GPU:

In [1]:
import torch

if not torch.cuda.is_available():
    raise RuntimeError("GPU not available")
else:
    print("GPU is available!")

GPU is available!


## Installing dependencies

In [2]:
%pip install --quiet https://github.com/abetlen/llama-cpp-python/releases/download/v0.3.4-cu122/llama_cpp_python-0.3.4-cp311-cp311-linux_x86_64.whl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 445.2/445.2 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.3 MB/s eta 0:00:00


In [3]:
%pip install --quiet ragatouille PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.9/490.9 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
 

In [4]:
%pip install --quiet git+https://github.com/mozilla-ai/structured-qa.git@5-add-benchmark

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 113.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 104.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 87.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.9 MB/s eta 0:00:00


In [5]:
!wget https://raw.githubusercontent.com/mozilla-ai/structured-qa/refs/heads/5-add-benchmark/benchmark/structured_qa.csv

--2025-02-04 18:45:44--  https://raw.githubusercontent.com/mozilla-ai/structured-qa/refs/heads/5-add-benchmark/benchmark/structured_qa.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23304 (23K) [text/plain]
Saving to: ‘structured_qa.csv’

structured_qa.csv   100%[===================>]  22.76K  --.-KB/s    in 0.001s  

2025-02-04 18:45:45 (15.7 MB/s) - ‘structured_qa.csv’ saved [23304/23304]



# Setup

In [6]:
import os

os.environ["LOGURU_LEVEL"] = "INFO"

In [7]:
from loguru import logger

In [8]:
import PyPDF2


def load_pdf(pdf_file: str) -> str | None:
    try:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        return "\n".join(page.extract_text() for page in pdf_reader.pages)
    except Exception as e:
        logger.exception(e)
        return None

## Function to Process all questions for a single Document

In [9]:
from ragatouille import RAGPretrainedModel
from ragatouille.data import CorpusProcessor


ANSWER_WITH_TYPE_PROMPT = """
You are a rigorous assistant answering questions.
You must only answer based on the current information available which is:

```
{CURRENT_INFO}
```

If the current information available not enough to answer the question,
you must return "I need more info" srting and nothing else:

If the current information is enough to answer, you must return one of the following formats:
- YES/NO (for boolean questions)
- Number (for numeric questions)
- Single letter (for multiple-choice questions)
"""


def process_document(
    document_file,
    document_data,
    model,
):
    logger.info("Setting up RAG")
    RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")
    corpus_processor = CorpusProcessor()
    documents = corpus_processor.process_corpus([load_pdf(document_file)])
    RAG.encode([x["content"] for x in documents])

    logger.info("Predicting")
    answers = {}
    sections = {}
    for index, row in document_data.iterrows():
        question = row["question"]
        question_part, *options = question.split("?")

        logger.info(f"Question: {question}")
        results = RAG.search_encoded_docs(query=question_part, k=3)
        current_info = "\n".join(result["content"] for result in results)
        logger.info(current_info[:100])

        messages = [
            {
                "role": "system",
                "content": ANSWER_WITH_TYPE_PROMPT.format(CURRENT_INFO=current_info),
            },
            {"role": "user", "content": question},
        ]
        answer = model.get_response(messages)
        logger.info(answer)
        answers[index] = answer
        sections[index] = None

    return answers, sections

## Load Model

In [10]:
from structured_qa.model_loaders import load_llama_cpp_model

In [11]:
model = load_llama_cpp_model(
    "bartowski/Qwen2.5-7B-Instruct-GGUF/Qwen2.5-7B-Instruct-Q8_0.gguf"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Qwen2.5-7B-Instruct-Q8_0.gguf:   0%|          | 0.00/8.10G [00:00<?, ?B/s]

# Run Benchmark

In [12]:
from pathlib import Path
from urllib.request import urlretrieve

import pandas as pd

logger.info("Loading input data")
data = pd.read_csv("structured_qa.csv")
data["pred_answer"] = [None] * len(data)
data["pred_section"] = [None] * len(data)
for document_link, document_data in data.groupby("document"):
    logger.info(f"Downloading document {document_link}")
    downloaded_document = Path(f"{Path(document_link).name}.pdf")
    if not Path(downloaded_document).exists():
        urlretrieve(document_link, downloaded_document)
        logger.info(f"Downloaded {document_link} to {downloaded_document}")
    else:
        logger.info(f"File {downloaded_document} already exists")

    answers, sections = process_document(downloaded_document, document_data, model)

    for index in document_data.index:
        data.loc[index, "pred_answer"] = str(answers[index]).upper()
        data.loc[index, "pred_section"] = sections[index]

data.to_csv("results.csv")

2025-02-04 18:50:10.763 | INFO     | __main__:<cell line: 0>:6 - Loading input data
2025-02-04 18:50:10.839 | INFO     | __main__:<cell line: 0>:11 - Downloading document https://aiindex.stanford.edu/wp-content/uploads/2024/05/HAI_AI-Index-Report-2024.pdf
2025-02-04 18:50:11.137 | INFO     | __main__:<cell line: 0>:15 - Downloaded https://aiindex.stanford.edu/wp-content/uploads/2024/05/HAI_AI-Index-Report-2024.pdf to HAI_AI-Index-Report-2024.pdf.pdf
2025-02-04 18:50:11.140 | INFO     | __main__:process_document:28 - Setting up RAG


artifact.metadata:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/405 [00:00<?, ?B/s]

...

2025-02-04 18:53:20.291 | INFO     | __main__:process_document:54 - 3
2025-02-04 18:53:20.292 | INFO     | __main__:process_document:41 - Question: How many victory conditions are there?
2025-02-04 18:53:20.311 | INFO     | __main__:process_document:44 - Conquering Middle-earth
If you are present in all 7 regions (with a Fortress and/or at least 1 Unit)
2025-02-04 18:53:20.923 | INFO     | __main__:process_document:54 - 3
2025-02-04 18:53:20.924 | INFO     | __main__:process_document:41 - Question: How many different races are there?
2025-02-04 18:53:20.943 | INFO     | __main__:process_document:44 - • 
 Re
veal tokens to both players. There is no hidden information.Bonus spaces
2 matching Race symb
2025-02-04 18:53:21.790 | INFO     | __main__:process_document:54 - I need more info
2025-02-04 18:53:21.791 | INFO     | __main__:process_document:41 - Question: Which player begins the game? -A: Sauron -B: The Fellowship -C: Other
2025-02-04 18:53:21.814 | INFO     | __main__:process_docu

Encoding 48 documents...


  0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()
100%|██████████| 2/2 [00:00<00:00, 10.62it/s]
2025-02-04 18:53:33.938 | INFO     | __main__:process_document:34 - Predicting
2025-02-04 18:53:33.942 | INFO     | __main__:process_document:41 - Question: What is the maximum number of cards a player may acquire during the lookout phase?
2025-02-04 18:53:33.958 | INFO     | __main__:process_document:44 - 13. At the beginning of the game, each player draws 5 cards from their 
Clan deck, looks through the


Shapes:
encodings: torch.Size([48, 508, 128])
doc_masks: torch.Size([48, 508])
Documents encoded!


2025-02-04 18:53:34.832 | INFO     | __main__:process_document:54 - 4
2025-02-04 18:53:34.834 | INFO     | __main__:process_document:41 - Question: Is there a limit to the number of cards a player may have in their hand?
2025-02-04 18:53:34.853 | INFO     | __main__:process_document:44 - NOTE 1:  There’s no limit to the number of cards a player may have 
in their hand.  
NOTE 2:  If the
2025-02-04 18:53:35.727 | INFO     | __main__:process_document:54 - NO
2025-02-04 18:53:35.728 | INFO     | __main__:process_document:41 - Question: Can you raid the locations of a player that has passed during the action phase?
2025-02-04 18:53:35.748 | INFO     | __main__:process_document:44 - Starting with the First player and continuing clockwise, 
each player performs one action at a time.
2025-02-04 18:53:36.585 | INFO     | __main__:process_document:54 - NO
2025-02-04 18:53:36.586 | INFO     | __main__:process_document:41 - Question: How many points in the scoreboard must be reached during the Ac

In [13]:
results = pd.read_csv("results.csv")
for index, result in results.iterrows():
    if result["pred_answer"].startswith(
        (f"-{result['answer']}", f"{result['answer']}")
    ):
        results.loc[index, "pred_answer"] = result["answer"]
results.loc[results["answer"] != results["pred_answer"]]

,Unnamed: 0,document,type,section,question,answer,pred_answer,pred_section
2,2,https://arxiv.org/pdf/1706.03762,Scientific Paper,3.1 Encoder and Decoder Stacks,How many layers compose the decoder?,6,N=6,NaN
10,10,https://arxiv.org/pdf/1706.03762,Scientific Paper,5.4 Regularization,What was the dropout rate used for the base mo...,0.1,PDROP= 0.1,NaN
28,28,https://arxiv.org/pdf/2201.11903,Scientific Report,3.1 Experimental Setup,How many large language models were evaluated?,5,FIVE,NaN
32,32,https://arxiv.org/pdf/2201.11903,Scientific Report,5 Symbolic Reasoning,Which symbolic reasoning task is used as an ou...,A,I NEED MORE INFO,NaN
34,34,https://arxiv.org/pdf/2201.11903,Scientific Report,3.2 Results,How many random samples were examined to under...,100,50,NaN
37,37,https://github.com/mozilla-ai/structured-qa/re...,Board Game,CARD AND TILE EFFECTS,How many different races are there?,6,I NEED MORE INFO,NaN
41,41,https://github.com/mozilla-ai/structured-qa/re...,Board Game,CHAPTER OVERVIEW,"After taking a landmark tile, do you reveal a ...",NO,YES,NaN
42,42,https://github.com/mozilla-ai/structured-qa/re...,Board Game,CARD AND TILE COSTS,Can a player pay coins to compensate for missi...,YES,NO,NaN
45,45,https://github.com/mozilla-ai/structured-qa/re...,Board Game,CARD AND TILE EFFECTS,Which type of cards provide coins? -A: Gray -B...,B,I NEED MORE INFO,NaN
55,55,https://github.com/mozilla-ai/structured-qa/re...,Board Game,EXPEDITION PHASE,Do you need a fish to conquer a distant island?,YES,NO,NaN


In [14]:
accuracy = sum(results["answer"] == results["pred_answer"]) / len(results)
accuracy

0.8349514563106796